# Introduction

In a prior notebook, documents were partitioned by assigning them to the domain with the highest Dice similarity of their term and structure occurrences. The occurrences of terms and structures in each domain is what we refer to as the domain "archetype." Here, we'll assess whether the observed similarity between documents and the archetype is greater than expected by chance. This would indicate that information in the framework generalizes well to individual documents.

# Load the data

In [1]:
import pandas as pd
import numpy as np
from utilities import *

In [2]:
framework = "dsm" 
version = 190325 # Document-term matrix version
suffix = "_opsim" # Suffix for term lists
n_iter = 100000 # Iterations for null distribution
dx = [0.33, 0.37, 0.31, 0.38, 0.37, 0.37, 0.38, 0.38] # Nudges for plotted means
ds = 0.115 # Nudges for plotted stars

## Brain activation coordinates

In [3]:
act_bin = load_coordinates()
print("Document N={}, Structure N={}".format(
      act_bin.shape[0], act_bin.shape[1]))

Document N=18155, Structure N=114


## Document-term matrix

In [4]:
dtm_bin = load_doc_term_matrix(version=version, binarize=True)
print("Document N={}, Term N={}".format(
      dtm_bin.shape[0], dtm_bin.shape[1]))

Document N=18155, Term N=4107


## Domain archetypes

In [5]:
from collections import OrderedDict

In [6]:
lists, circuits = load_framework(framework, suffix=suffix)

In [7]:
words = sorted(list(set(lists["TOKEN"])))
structures = sorted(list(set(act_bin.columns)))
domains = list(OrderedDict.fromkeys(lists["DOMAIN"]))

In [8]:
archetypes = pd.DataFrame(0.0, index=words+structures, columns=domains)
for dom in domains:
    for word in lists.loc[lists["DOMAIN"] == dom, "TOKEN"]:
        archetypes.loc[word, dom] = 1.0
    for struct in structures:
        archetypes.loc[struct, dom] = circuits.loc[struct, dom]
archetypes[archetypes > 0.0] = 1.0
print("Term & Structure N={}, Domain N={}".format(
      archetypes.shape[0], archetypes.shape[1]))

Term & Structure N=196, Domain N=9


## Document structure-term vectors

In [9]:
pmids = dtm_bin.index.intersection(act_bin.index)
len(pmids)

18155

In [10]:
dtm_words = dtm_bin.loc[pmids, words]
act_structs = act_bin.loc[pmids, structures]

In [11]:
docs = dtm_words.copy()
docs[structures] = act_structs.copy()
docs.head()

,acute_stress_disorder,adhd,adjustment_disorder,agoraphobia,alcohol_abuse,alcohol_use_disorder,alcohol_withdrawal,anxiety_disorder,aspergers_syndrome,attention_deficit_hyperactivity_disorder,...,right_superior_temporal_gyrus_posterior_division,right_supplementary_motor_cortex,right_supracalcarine_cortex,right_supramarginal_gyrus_anterior_division,right_supramarginal_gyrus_posterior_division,right_temporal_fusiform_cortex_anterior_division,right_temporal_fusiform_cortex_posterior_division,right_temporal_occipital_fusiform_cortex,right_temporal_pole,right_thalamus
10022492,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,1,0,0,0,0,0,1
10022494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,0,0,0,1,0,0
10022496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,0,0,0,0
10027301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
10051677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,1,0,1,1


## Document assignments

In [12]:
doc2dom_df = pd.read_csv("../partition/data/doc2dom_{}.csv".format(framework), 
                         header=None, index_col=0)
doc2dom = {int(pmid): int(dom) for pmid, dom in doc2dom_df.iterrows()}

In [13]:
dom2docs = {dom: [] for dom in domains}
for doc, dom in doc2dom.items():
    dom2docs[domains[dom-1]].append(doc)

# Compute similarity to archetype

In [14]:
from scipy.spatial.distance import cdist

## Observed values

In [15]:
df_obs = pd.DataFrame(index=domains, columns=pmids)
for dom in domains:
    dom_pmids = dom2docs[dom]
    dom_vecs = docs.loc[dom_pmids].values
    dom_arche = archetypes[dom].values.reshape(1, archetypes.shape[0])
    dom_sims = 1.0 - cdist(dom_vecs, dom_arche, metric="dice")
    df_obs.loc[dom, dom_pmids] = dom_sims[:,0]
df = pd.DataFrame({"OBSERVED": df_obs.mean(axis=1)}, index=domains)
df.head(8)

,OBSERVED
DEVELOPMENTAL,0.377670
PSYCHOTIC,0.211560
BIPOLAR,0.164795
DEPRESSIVE,0.163331
ANXIETY,0.270181
OBSESSIVE_COMPULSIVE,0.199464
TRAUMA_STRESSOR,0.278839
DISRUPTIVE,0.055618


## Null distributions

In [16]:
import os

In [17]:
file_null = "data/arche_null_{}_{}iter.csv".format(framework, n_iter)
if not os.path.isfile(file_null):
    df_null = np.zeros((len(domains), n_iter))
    for n in range(n_iter):
        null = np.random.choice(range(len(docs.columns)), 
                                size=len(docs.columns), replace=False)
        for i, dom in enumerate(domains):
            dom_pmids = dom2docs[dom]
            dom_vecs = docs.loc[dom_pmids].values
            dom_arche = archetypes.values[null,i].reshape(1, archetypes.shape[0])
            df_null[i,n] = 1.0 - np.mean(cdist(dom_vecs, dom_arche, metric="dice"))
        if n % int(n_iter / 10.0) == 0:
            print("Processed {} iterations".format(n))
    df_null = pd.DataFrame(df_null, index=domains, columns=range(n_iter))
    df_null.to_csv(file_null)
else:
    df_null = pd.read_csv(file_null, index_col=0, header=0)

# Perform significance testing

In [18]:
from statsmodels.stats import multitest

In [19]:
alphas = [0.0001, 0.00001, 0.000001]

In [20]:
pval = []
for dom in domains:   
    dom_null = df_null.loc[dom].values
    dom_obs = float(df.loc[dom, "OBSERVED"])
    p = np.sum(dom_null > dom_obs) / float(n_iter)
    pval.append(p)
    df.loc[dom, "P"] = p
df["FDR"] = multitest.multipletests(pval, method="fdr_bh")[1]
for dom in domains:
    df.loc[dom, "STARS"] = ""
    for star, alpha in zip(["*", "**", "***"], alphas):
        if df.loc[dom, "FDR"] < alpha:
            df.loc[dom, "STARS"] = star
df = df.loc[domains, ["OBSERVED", "P", "FDR", "STARS"]]
df

KeyError: 'DISRUPTIVE'

# Plot results

In [ ]:
%matplotlib inline

In [ ]:
palette = palettes[framework]

In [ ]:
plot_archetypes(framework, domains, df, df_null, df_obs, palette, 
                dx=dx, ds=ds, alphas=alphas, interval=0.999)